# Automation

## Introduction

Condition Based Monitoring, sometimes called Edge Network Automation, is the idea that edge sensors and actuators should not be responsible for decision making or coordinating the responses to various edge network condition. However, neither should all of this decision making and analytics processing be performed in the cloud. The IoT Gateway should be remotely programmable by an IoT administrator or software developer in such a way that local events can be automatically managed and reported. Intelligent closed loop systems are able to coordinate responses to conditions on their own networks and report back to the cloud or a network operation center.

In the example of a temperature controlled room, the temperature sensor is reporting the temperature to the Intel® IoT Gateway and the gateway is responsible for triggering an IoT event. An IoT Event (also called a trigger in this workshop) always has a conditions function and a trigger function. In the case that the temperature is too hot the event may take automatic action to turn on the air conditioning on and send an alert to the person responsible for the room.



## Purpose of the Lab

In this lab, we will implement a Python based rules engine to automate IoT action on a network. This system can be used to build a smart home, a smart factory or other Internet of Things installations.

The student should note that this implementation's primary goal is to teach the fundementals of implementing a rules based IoT automation system. 

There are many other considerations in building a more production ready environment. A production environment should consider the actors of the system such as the system administrators, business policy makers, factory workers, developers, etc ... It should also consider integration with the pre-existing software environment, enhanced security measures and building a human interface device and graphical user interface for the operators.

This lab's purpose is to focus on the core ideas and not to distract with the many ways that it could be built out or integrated into other enviroments.

## Objectives

By the end of this module, you should be able to:

* Implement your automation rules in the form of conditional actions called triggers that run on the edge network.
* Read triggers from the database and evaluate whether any of them should be executed.
* Write your own Automation Service




## Prerequisites and Architecture Overview

As we begin this lab, we assume that the developer has an Intel IoT device on the network and that there are **sensors on the network transmitting data over MQTT**.

Our automation service will start by reading in a list of automation rules and listening to incoming MQTT sensor data.

A Rule is defined to have a Name, and Sensor that it monitors, a true/false predicate function to determine whether the rule should be run and a function action to run if the predicate is true.

To build this automation service, we will start by: 
1. **Defining a Rule Class** and creating several examples to get the student familiar with them. 
2. Secondly, we will **define several helper functions** that will allow us to use a function programming style to filter the list of Rules. When data comes in to the automation service, we will want to filter the list of Rules so that only rules that apply to incoming data are checked and evaluated.
3. **Create a simple Python service to listen for MQTT**
4. Use the **helper functions to filter the list of rules**.
5. **Execute every rule whose predicate function evaluates to true**.


## Implementing a Rule

### The Rule Class

Here is the definition of a Rule base class. To define a rule you will need to extend this class. 

This base class provides:
* a **constructor function** that assigns the rule to a unique sensor on the network. 
* default **predicate function** which returns always False. This means that by default the action function is not never called. This function should be overridden in a derived class.
* an **__do function** that calls the predicate function and if it returns true then calls the action function
* an **action function** to perform the main task. This function should be overridden in a derived class.

Note that the __\_\_do function__ is a private function that can not be accessed outside of the class.

In [14]:
import sys
class Rule:
    """
    A Base Class for defining IoT automation rules.
    """
    
    def __init__(self, sensorID):
        """
        Constructor function that takes a id 
        that uniquely identifies a sensor.
        """
        self.sensorID = sensorID
    
    def predicate(self, sensorValue):
        "In the base Rule class, the predicate always returns False"
        return False
    
    def __do(self, sensorValue):
        try:
            if self.predicate(sensorValue):
                self.action(sensorValue)
        except:
            print "Unexpected error:", sys.exc_info()[0]
                    
    def action(self, sensorValue):
        print("Generic Rule activiate on " + 
              self.sensorID + " with senor value " + str(sensorValue))    

### Define a Subclass Rule Class that can be Instantiated

Now that we've defined the base Rule class, we can show examples of how to use it to derive a specific rule class.

The sky is the limit with the action that you choose to define. Some common action might be to sent a text message to a list of system administrators when something isn't working properly, or to log a message to a database. You can include any Python libraries that you wish when defining these functions.

Let define a simple Rule that prints to the console when the temperature rises over 20C. 

In [17]:
class TemperatureOver20Rule(Rule):
    def predicate(self, sensorValue):
        return sensorValue > 20
    
    def action(self, sensorValue):
        print("Temperature Over 20 Rule activated on " + self.sensorID + " with senor value " + str(sensorValue))

And another rule that prints to the console if the temperature falls below 15C.

In [18]:
class TemperatureUnder15Rule(Rule):
    def predicate(self, sensorValue):
        return sensorValue < 15
    
    def action(self, sensorValue):
        print("Temperature Under 15 Rule activated on " + 
              self.sensorID + 
              " with senor value " + 
              str(sensorValue))
        

### Instantiate the Rules

In [19]:
r1 = Rule("temperature1")
r2 = TemperatureUnder15Rule("temperature")
r3 = TemperatureOver20Rule("temperature")
r4 = Rule("temperature4")

In [20]:
rules = r1, r2, r3, r4

Return an array of results from each rules predicate function

### Define Helper Functions

The first helper function that we will define will take the entire list of automation rules and filter it based on sensorID. Becasue we are listening to JSON object that contain a **sensorID**, a **value** and a **timestamp**, this enables us to filter the automation rules to only include the rules that apply to the sensorID of the incoming data.

In [21]:
def filterBySensorId(sensorID, rules):
    "Filter a list of rules by sensorID"
    return [rule for rule in rules if rule.sensorID == sensorID]

The second filter that we will define will take a list of automation rules and execute each of their **predicate** functions and return a list of rules that should be executed on the incoming data.

In [22]:
def filterByPredicate(sensorData, rules):
    "Filter a list of rules by its predicate"
    [rule for rule in rules if rule.predicate(sensorData) == True]

In [23]:
print filterByPredicate(20, rules)

None


Let's test this helper to verify that it works as expected.

In [24]:
filterBySensorId("temperature1", rules)

[<__main__.Rule instance at 0x10cd53560>]

In [25]:
filterBySensorId("temperature2", rules)

[]

In [26]:
filterBySensorId("temperature3", rules)

[]

In [27]:
filterBySensorId("temperature4", rules)

[<__main__.Rule instance at 0x10cdb1320>]

In [28]:
[r.predicate(20) for r in rules]

[False, False, False, False]

In [29]:
[r.action(21) for r in rules]

Generic Rule activiate on temperature1 with senor value 21
Temperature Under 15 Rule activated on temperature with senor value 21
Temperature Over 20 Rule activated on temperature with senor value 21
Generic Rule activiate on temperature4 with senor value 21


[None, None, None, None]

## Helper Functions to Encode/Decode JSON

In [3]:
import json

In [8]:
sampleData = '''
{
  "sensor_id":"temperature3",
  "value":17,
  "timestamp":1513807710949
}
            '''
print(sampleData)


{
  "sensor_id":"temperature3",
  "value":17,
  "timestamp":1513807710949
}
            


In [9]:
try:
    parsed_json = json.loads(sampleData)
except:
     print("Unexpected error:", sys.exc_info()[0])

In [10]:
parsed_json['value']

17

In [11]:
[r.action(parsed_json['value']) 
  for r in filterBySensorId(
                  parsed_json['sensor_id'], rules
  )]

NameError: name 'filterBySensorId' is not defined

## Setup the MQTT subscription of the Automation Service

In [21]:
import paho.mqtt.client as mqtt

In [22]:
def on_connect(mqttc, obj, flags, rc):
    print("rc: " + str(rc))

In [23]:
def on_message(mqttc, obj, msg):
    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    try:
        parsed_json = json.loads(msg.payload)
    except:
        print("Unexpected error:", sys.exc_info()[0])
        
    [r.action(parsed_json['value']) for r in filterBySensorId(parsed_json['sensor_id'], rules)]

In [24]:
def on_subscribe(mqttc, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

In [25]:
def on_log(mqttc, obj, level, string):
    print(string)

In [26]:
mqttc = mqtt.Client()
mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_subscribe = on_subscribe

# Uncomment to enable debug messages
# mqttc.on_log = on_log

In [27]:
mqttc.connect("localhost", 1883, 60)

0

In [ ]:
mqttc.subscribe("sensors/temperature/data")

(0, 1)

In [ ]:
mqttc.loop_forever()

rc: 0
Subscribed: 1 (0,)
sensors/temperature/data 0 {"sensor_id":"temperature","value":24,"timestamp":1513811032566}

Temperature Over 20 Rule activated on temperature with senor value 24

sensors/temperature/data 0 {"sensor_id":"temperature","value":21,"timestamp":1513811033570}

Temperature Over 20 Rule activated on temperature with senor value 21

sensors/temperature/data 0 {"sensor_id":"temperature","value":19,"timestamp":1513811034572}
sensors/temperature/data 0 {"sensor_id":"temperature","value":21,"timestamp":1513811035573}

Temperature Over 20 Rule activated on temperature with senor value 21

sensors/temperature/data 0 {"sensor_id":"temperature","value":24,"timestamp":1513811036574}

Temperature Over 20 Rule activated on temperature with senor value 24

sensors/temperature/data 0 {"sensor_id":"temperature","value":15,"timestamp":1513811037576}
sensors/temperature/data 0 {"sensor_id":"temperature","value":17,"timestamp":1513811038578}
sensors/temperature/data 0 {"sensor_id":"te